#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("/Users/chira/Desktop/NECESSITY/Data science/Langchain/data_ingestion/speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [2]:
docs

[Document(metadata={'source': '/Users/chira/Desktop/NECESSITY/Data science/Langchain/data_ingestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': '/Users/chir

In [12]:
embeddings=(
    OllamaEmbeddings(model='llama3.1')
)

In [14]:
db=FAISS.from_documents(docs,embeddings)
db

In [15]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content


'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [16]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [17]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(metadata={'source': '/Users/chira/Desktop/NECESSITY/Data science/Langchain/data_ingestion/speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  18410.371),
 (Document(metadata={'source': '/Users/chira/Desktop/NECESSITY/D

In [18]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[0.9388034343719482,
 -3.2526724338531494,
 4.532981872558594,
 1.021759033203125,
 -2.318324327468872,
 -0.3971823453903198,
 4.455745220184326,
 0.39895737171173096,
 0.4291878938674927,
 2.214548349380493,
 1.5728281736373901,
 1.6477861404418945,
 0.9818585515022278,
 3.4852256774902344,
 1.0710610151290894,
 1.428680419921875,
 -3.2284395694732666,
 2.116316080093384,
 -1.4524881839752197,
 0.09688194841146469,
 -0.9532478451728821,
 -0.5811517238616943,
 1.76360023021698,
 0.25203055143356323,
 1.0012500286102295,
 0.00982540100812912,
 0.27192243933677673,
 -2.212578058242798,
 -1.5102083683013916,
 -0.4448060393333435,
 2.093620777130127,
 -0.9274644255638123,
 1.695320725440979,
 3.453007221221924,
 7.010448932647705,
 -0.9892572164535522,
 -2.3044145107269287,
 -0.16184017062187195,
 2.6324141025543213,
 1.6977957487106323,
 3.964254140853882,
 -0.47199010848999023,
 1.4789546728134155,
 -0.41951286792755127,
 0.42426878213882446,
 -0.984699010848999,
 0.5224863886833191,
 -0

In [19]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(metadata={'source': '/Users/chira/Desktop/NECESSITY/Data science/Langchain/data_ingestion/speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(metadata={'source': '/Users/chira/Desktop/NECESSITY/Data science/Lang

In [20]:
### Saving And Loading
db.save_local("faiss_index")

In [21]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [22]:
docs

[Document(metadata={'source': '/Users/chira/Desktop/NECESSITY/Data science/Langchain/data_ingestion/speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(metadata={'source': '/Users/chira/Desktop/NECESSITY/Data science/Lang